In [ ]:
from google.colab import drive
drive.mount('/content/drive')
noReg='/content/drive/MyDrive/myProject/model.h5'
reg2Link='/content/drive/MyDrive/myProject/reg2model.h5' 
reg1Link='/content/drive/MyDrive/myProject/reg1model.h5'

#Set link to h5 model
link=reg2Link

Mounted at /content/drive


In [ ]:
import tensorflow as tf
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import time
import copy
import tensorflow_hub as hub
import matplotlib
from tensorflow.python.keras import regularizers
from keras.regularizers import l1
from keras.regularizers import l2
from keras.layers import InputLayer

x_train = y_train = x_test = y_test = None
num_classes = 10
input_shape = (28, 28, 1)
batch_size = 128
epochs = 1
epsilon = 0.05
success = 0
shots = 10
# alpha=0.38240
alpha = np.exp(-2)
stream_length = 10000
select = []
result = None
ground_truth = None
vals = []
online = None
mother_result = None
mother_stream = None
mother_ground_truth = None
mother_flag = False
mother_initial_probability = None
mother_friendly_probability = None
mother_real_loss=None
real_loss=None
friendly_probability=None
initial_probability=None
secretary_chosen=[]
hard_chosen=[]
random_success=[]
secretary_success=[]
HLGRatio=[]
LLGRatio=[]
midsuc=0
mlg=0
fr=[]
indices=[]
model = None
indices = []
s = 0
hard = 0
tile=4
reg_strength=1
reg_type="l2"

if reg_type=='l2':
  link=reg2Link

elif reg_type=='l1':
  link=reg1Link

else:
  link=None






friendlyEpsilon = 0.3
reg_loss = None


#Calculate ground truth of regularization loss of the model
def calcReg(strength,mode='l1'): #other l2
  global reg_loss,model
  reg_loss=0
  
  for layer in model.layers:
    for temp in layer.weights:
      A=np.asarray(temp)

      added=None

      if mode=='l1':
        added=np.sum(A)

      elif mode=='l2':
        A=np.power(A,2)
        added=np.sum(A)
      
      else:
        raise('error')
      
      reg_loss=reg_loss+added

    
    reg_loss=reg_loss*strength
    

      








class element:
    def __init__(self, i, value):
        self.index = i
        self.v = value

    def __ge__(self, other):
        return self.v > other.v

    def __lt__(self, other):
        return self.v < other.v


def testImage(clean_img, adv_image, truth, pred):
    plt.imshow(adv_image - clean_img, cmap='gray', vmin=-epsilon, vmax=1)
    plt.show()

def prepareDataset():
    global x_train, y_train, x_test, y_test
    (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
    x_train = x_train.astype("float32") / 255
    x_test = x_test.astype("float32") / 255
    x_train = np.expand_dims(x_train, -1)
    x_test = np.expand_dims(x_test, -1)
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)

def defineModel():
    

    global model
    reg=None

    if reg_type=='l1':
      reg=l1(reg_strength)

    elif reg_type=='l2':
      reg=l2(reg_strength)


    model = Sequential()
    model.add(InputLayer(input_shape=input_shape))
    model.add(layers.Conv2D(64, kernel_size=(5, 5), activation="relu",kernel_regularizer=reg))
    model.add(layers.Conv2D(64, kernel_size=(5, 5), activation="relu",kernel_regularizer=reg))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation="relu",kernel_regularizer=reg))
    model.add(layers.Dense(num_classes, activation="softmax",kernel_regularizer=reg))
    model.build()



def trainModel():
    global model

    model.load_weights(link)
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    calcReg(reg_strength,reg_type)
    print(reg_loss,'reg_loss')
    model.summary()
      


def evalModel():
    score = model.evaluate(x_test, y_test, verbose=0)
    print("Test loss:", score[0])
    print("Test accuracy:", score[1])

#Perturbs image
# other possible mode : friendly
def perturb(idx,mode='adversarial'): # other possible mode : friendly,duo. duo is used for speed up and epsilon is used as friendlyEpsilon

  y=np.expand_dims(y_test[idx],axis=0)
  temp=np.squeeze(x_test[idx])
  x=copy.deepcopy(temp)
  highest_loss_found=None
  lowest_loss_found=None
  highest_perturbation=None
  lowest_perturbation=None

  

  cce = tf.keras.losses.CategoricalCrossentropy()
  perturbation=np.zeros((28,28))
  num=int(28/tile)

  if mode=='adversarial':

    for i in range(num):
      for j in range(num):
        plus=copy.deepcopy(x)
        minus=copy.deepcopy(x)

        plus[i*tile:(i+1)*tile , j*tile:(j+1)*tile]=  plus[i*tile:(i+1)*tile , j*tile:(j+1)*tile]+epsilon
        minus[i*tile:(i+1)*tile , j*tile:(j+1)*tile]=  minus[i*tile:(i+1)*tile , j*tile:(j+1)*tile]-epsilon


        plus=np.expand_dims(plus,axis=(0,-1))
        minus=np.expand_dims(minus,axis=(0,-1))
        plus=model(plus)
        minus=model(minus)
        plus=cce(y,plus).numpy()
        minus=cce(y,minus).numpy()
        
        if plus>minus:
          perturbation[i*tile:(i+1)*tile , j*tile:(j+1)*tile]=epsilon
          x[i*tile:(i+1)*tile , j*tile:(j+1)*tile]+=epsilon

          if highest_loss_found==None or plus>highest_loss_found:
            highest_loss_found=plus
            highest_perturbation=copy.deepcopy(perturbation)



        else:
          perturbation[i*tile:(i+1)*tile , j*tile:(j+1)*tile]=-epsilon
          x[i*tile:(i+1)*tile , j*tile:(j+1)*tile]-=epsilon

          if highest_loss_found==None or minus>highest_loss_found:
              highest_loss_found=minus
              highest_perturbation=copy.deepcopy(perturbation)
              

    return highest_perturbation



  elif mode=='friendly':

    for i in range(num):
      for j in range(num):
        plus=copy.deepcopy(x)
        minus=copy.deepcopy(x)

        plus[i*tile:(i+1)*tile , j*tile:(j+1)*tile]=  plus[i*tile:(i+1)*tile , j*tile:(j+1)*tile]+friendlyEpsilon
        minus[i*tile:(i+1)*tile , j*tile:(j+1)*tile]=  minus[i*tile:(i+1)*tile , j*tile:(j+1)*tile]-friendlyEpsilon


        plus=np.expand_dims(plus,axis=(0,-1))
        minus=np.expand_dims(minus,axis=(0,-1))
        plus=model(plus)
        minus=model(minus)
        plus=cce(y,plus).numpy()
        minus=cce(y,minus).numpy()
        
        if plus>minus:
          perturbation[i*tile:(i+1)*tile , j*tile:(j+1)*tile]=-friendlyEpsilon
          x[i*tile:(i+1)*tile , j*tile:(j+1)*tile]-=friendlyEpsilon

          if lowest_loss_found==None or minus<lowest_loss_found:
            lowest_loss_found=minus
            lowest_perturbation=copy.deepcopy(perturbation)



        else:
          perturbation[i*tile:(i+1)*tile , j*tile:(j+1)*tile]=friendlyEpsilon
          x[i*tile:(i+1)*tile , j*tile:(j+1)*tile]+=friendlyEpsilon

          if lowest_loss_found==None or plus<lowest_loss_found:
              lowest_loss_found=plus
              lowest_perturbation=copy.deepcopy(perturbation)

    return lowest_perturbation


  else:
      raise Exception("invalid mode")
      


#Assess the metric for the index corresponding to image
def assess(index, metric="Loss"):
    global success
    x = np.expand_dims(x_test[index], axis=0)
    y_expanded = np.expand_dims(y_test[index], axis=0)


    cce = tf.keras.losses.CategoricalCrossentropy()
    y=model(x)
  
    initLoss= cce(y_expanded, y)


    perturbation = perturb(index,'adversarial')
    perturbation=np.expand_dims(perturbation,axis=(0,-1))

    xp = x + perturbation
    y = model(xp)
    pr = np.argmax(y)
    successed = True

    if pr == np.argmax(y_test[index]):
        successed = False

    
    loss = cce(y_expanded, y)
    return (loss+reg_loss, successed, initLoss+reg_loss,initLoss+reg_loss,loss)#friendlyLoss+reg_loss


#This function attacks all dataset and caches the value. To generate a stream, it permutates the indices of the test set
#So we won't need to compute attack for each new stream
def generate_stream_data():
    global result, ground_truth, mother_result, mother_stream, mother_ground_truth, mother_flag, mother_initial_probability, mother_friendly_probability,friendly_probability,initial_probability,mother_real_loss,real_loss,defsuc,deffail,midsuc,mlg
   

    lower = -np.log10(0.5)
    upper = -np.log10(10)

    defsuc = 0
    midsuc=0
    deffail = 0

    if mother_flag == False:
        mother_stream = np.random.choice(10000, size=stream_length, replace=False)
        mother_ground_truth = np.zeros((stream_length))
        mother_result = np.zeros((stream_length))
        mother_flag = True
        mother_initial_probability = np.zeros((stream_length))
        mother_friendly_probability = np.zeros((stream_length))
        mother_real_loss=np.zeros((stream_length))
        real_loss=np.zeros((stream_length))

        # (loss,successed,initProb,friendProb)
        for i in range(stream_length):
            if i % 100 == 0:
                print(i / 100, "%")

            mother_ground_truth[i], mother_result[i], mother_initial_probability[i], mother_friendly_probability[i],mother_real_loss[i] = assess(mother_stream[i])
            if mother_real_loss[i] < lower:
                deffail += 1



            if mother_real_loss[i] > upper:
                defsuc += 1
       

            if mother_real_loss[i]>lower and mother_real_loss[i]<upper and mother_result[i]:
              midsuc+=1
        mlg=stream_length-defsuc-deffail
        print("defsuc : ",defsuc)
        print("midsuc : ",midsuc)
        print("len mlg : ",mlg)
        print("deffail : ",deffail)

    
    idx = np.random.permutation(np.arange(stream_length))

    result = mother_result[idx]
    ground_truth = mother_ground_truth[idx]
    friendly_probability=mother_friendly_probability[idx]
    initial_probability=mother_initial_probability[idx]
    real_loss=mother_real_loss[idx]



#The main criteria algorithm
def onlineAttack(verbose=False):
  global ground_truth,vals,s,indices,hard,initial_probability,friendly_probability
  t=int(alpha*stream_length)
  R=[]
  choosen_sum=0
  indices=[]
  vals=[]
  hard=0

  lower=0.6931471824645996 #-log10(0.5)
  upper=2.3025851249694824 #-log10(1/10)
  remainedShots=shots
  best_estimated_regularization=1000000


  for i in range(stream_length):
      best_estimated_regularization=min(friendly_probability[i],initial_probability[i],best_estimated_regularization)
      ground_truth[i]=ground_truth[i]-best_estimated_regularization
  
      if verbose:
        print("best estimation : ",best_estimated_regularization," current friendly loss : ",friendly_probability[i]," current ce loss : ",ground_truth[i]," init probability : ",initial_probability[i],"real loss : ",real_loss[i])

  for i in range (t):
    if len(indices)>=shots:
      break
    if ground_truth[i]<lower:

      continue
    if ground_truth[i]>upper:
      hard+=1
      remainedShots-=1
      indices.append(i)

      while len(R)>remainedShots:
        R.pop(0)
        
        

      continue

    if len(R)<remainedShots:
      R.append(ground_truth[i])

    elif len(R)==remainedShots:
      R.sort()

    else:
      if ground_truth[i]>R[0]:
        R[0]=ground_truth[i]
        R.sort()
  

  for i in range(t,stream_length):
    if len(indices)>=shots:
      break

    if ground_truth[i]<lower:
      continue

    if ground_truth[i]>upper: 
      indices.append(i)
      remainedShots-=1
      hard+=1
      while len(R)>remainedShots and len(R)>0:
        R.pop(0)
        ("pre len(R)=",len(R),"len(indices):",len(indices))
      continue
    

   
    if len(R)>0 and ground_truth[i]>R[0]:
      indices.append(i)
      R[0]=copy.deepcopy(ground_truth[i])
      choosen_sum+=R[0]
      vals.append(copy.deepcopy(R[0]))
      R.sort()
      continue

    if len(R)==0:
      indices.append(i)
      R.append(copy.deepcopy(ground_truth[i]))
      choosen_sum+=R[0]
      vals.append(copy.deepcopy(R[0]))


    
    if stream_length-i == shots - len(indices):
      for j in range (i,stream_length):
        if ground_truth[j]<lower:
          continue
        indices.append(j)
        vals.append(copy.deepcopy(ground_truth[j]))
        choosen_sum+=ground_truth[j]
      break

  s=0

  for i in indices:
    if result[i]:
      s+=1
  

  hard_chosen.append(hard)
  secretary_chosen.append(len(indices)-hard)
  
  fr.append(s/shots)



fool_rate=[]
success_avg=[]
success_std=[]
fail_avg=[]
fail_std=[]
total_avg=[]
total_std=[] 



def plot(show=False):
  global result,ground_truth,vals,hard,indices,hard_chosen,secretary_chosen,secretary_success,fr,real_loss,HLGRatio,MLGRatio
  lower=0.6931471824645996
  upper=2.3025851249694824
  mlg_success=0

  for i in indices:
   
  
    if ground_truth[i]<= upper and ground_truth[i]>=lower and result[i] :
      mlg_success+=1

  if secretary_chosen[-1]>0:
    secretary_success.append(mlg_success/secretary_chosen[-1])

  mysuc=0
  myfail=0

  for i in range(stream_length):
    if ground_truth[i]>=upper:
      mysuc+=1
    
    if ground_truth[i]<=lower:
      myfail+=1

  HLGRatio.append(mysuc)
  LLGRatio.append(myfail)
      


def randomSelectionInMLG():
  mlg=0
  success=0
  lower=0.6931471824645996
  upper=2.3025851249694824

  for i in range(stream_length):
     if ground_truth[i]<= upper and ground_truth[i]>=lower:
       mlg+=1

       if result[i]:
         success+=1
  
  return success/mlg


def randomSelectionNotKnowingM():
  total=0
  success=0
  lower=0.6931471824645996
  upper=2.3025851249694824

  for i in range(stream_length):
     if ground_truth[i]>=lower:
       total+=1
       
       if result[i]:
         success+=1


  return success/total


#########################################################################################################################

prepareDataset()
defineModel()
trainModel()
evalModel()

c = 0
generate_stream_data()
onlineAttack()

















910.3044008939178 reg_loss
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
module_wrapper_10 (ModuleWra (None, 24, 24, 64)        1664      
_________________________________________________________________
module_wrapper_11 (ModuleWra (None, 20, 20, 64)        102464    
_________________________________________________________________
module_wrapper_12 (ModuleWra (None, 25600)             0         
_________________________________________________________________
module_wrapper_13 (ModuleWra (None, 128)               3276928   
_________________________________________________________________
module_wrapper_14 (ModuleWra (None, 10)                1290      
Total params: 3,382,346
Trainable params: 3,382,346
Non-trainable params: 0
_________________________________________________________________
Test loss: 0.043548583984375
Test accuracy: 0.9854000210762024
0.0 %
1.0 %
2.0 %
3.

In [ ]:
#obtain attack results for 1000 iterations with k=100

hard_chosen=[]
secretary_chosen=[]
secretary_success=[]
fr=[]
shots=100
iter=1000


for i in range(iter):
  generate_stream_data()
  onlineAttack()
  plot()



fr=np.asarray(fr, dtype=np.float32)
secretary_success=np.asarray(secretary_success, dtype=np.float32)

secretary_chosen=np.asarray(secretary_chosen, dtype=np.float32)
hard_chosen=np.asarray(hard_chosen, dtype=np.float32)

print("These are general results yielded for ",iter," number of random permutations")
print("average fool rate under my assumption : ",np.mean(fr))

print("average success for secretary chosen data : ",np.mean(secretary_success))
print("average number of data chosen by secretary : ",np.mean(secretary_chosen))
print("average number of hard chosen data : ",np.mean(hard_chosen))
print("average number of selected data : ",np.mean(hard_chosen+secretary_chosen))
print("random MLG success rate : ",randomSelectionInMLG())
print("random choice not knowing m success rate : ",randomSelectionNotKnowingM())
print("mean HLG grouped : ",np.mean(np.asarray(HLGRatio)))
print("mean LLG grouped : ",np.mean(np.asarray(LLGRatio)))


hard_chosen=[]
secretary_chosen=[]

secretary_success=[]
fr=[]



def plotShots():
  
  for i in indices:
    x= np.expand_dims(x_test[i], axis=0)
    yc=model(x)
    perturbation=fgsm(epsilon,i)
    print("perturbation shape ",perturbation.shape)
    print("x",x.shape)
    perturbation=np.expand_dims(perturbation,axis=(0,-1))
    xp=np.squeeze(perturbation)+np.squeeze(x)
    xp=np.expand_dims(xp,axis=(0,-1))
  
    yp=model(xp)

    if (np.argmax(yc)!=np.argmax(yp)) and (result[i] != -1):
      testImage(np.squeeze(x),np.squeeze(xp),np.argmax(yc),np.argmax(yp))




These are general results yielded for  1000  number of random permutations
average fool rate under my assumption :  0.9952899
average success for secretary chosen data :  0.985906
average number of data chosen by secretary :  32.437
average number of hard chosen data :  67.563
average number of selected data :  100.0
random MLG success rate :  0.9126819126819127
random choice not knowing m success rate :  0.9433962264150944
mean HLG grouped :  260.9925
mean LLG grouped :  9258.0045


In [ ]:
#obtain attack results for 1000 iterations with k=10

hard_chosen=[]
secretary_chosen=[]
secretary_success=[]
fr=[]
shots=10
iter=1000


for i in range(iter):
  generate_stream_data()
  onlineAttack()
  plot()



fr=np.asarray(fr, dtype=np.float32)
secretary_success=np.asarray(secretary_success, dtype=np.float32)

secretary_chosen=np.asarray(secretary_chosen, dtype=np.float32)
hard_chosen=np.asarray(hard_chosen, dtype=np.float32)

print("These are general results yielded for ",iter," number of random permutations")
print("average fool rate under my assumption : ",np.mean(fr))

print("average success for secretary chosen data : ",np.mean(secretary_success))
print("average number of data chosen by secretary : ",np.mean(secretary_chosen))
print("average number of hard chosen data : ",np.mean(hard_chosen))
print("average number of selected data : ",np.mean(hard_chosen+secretary_chosen))
print("random MLG success rate : ",randomSelectionInMLG())
print("random choice not knowing m success rate : ",randomSelectionNotKnowingM())
print("mean HLG grouped : ",np.mean(np.asarray(HLGRatio)))
print("mean LLG grouped : ",np.mean(np.asarray(LLGRatio)))


hard_chosen=[]
secretary_chosen=[]

secretary_success=[]
fr=[]


def plotShots():
  
  for i in indices:
    x= np.expand_dims(x_test[i], axis=0)
    yc=model(x)
    perturbation=fgsm(epsilon,i)
    print("perturbation shape ",perturbation.shape)
    print("x",x.shape)
    perturbation=np.expand_dims(perturbation,axis=(0,-1))
    xp=np.squeeze(perturbation)+np.squeeze(x)
    xp=np.expand_dims(xp,axis=(0,-1))
  
    yp=model(xp)

    if (np.argmax(yc)!=np.argmax(yp)) and (result[i] != -1):
      testImage(np.squeeze(x),np.squeeze(xp),np.argmax(yc),np.argmax(yp))

#plotShots()



These are general results yielded for  1000  number of random permutations
average fool rate under my assumption :  1.0
average success for secretary chosen data :  nan
average number of data chosen by secretary :  0.0
average number of hard chosen data :  10.0
average number of selected data :  10.0
random MLG success rate :  0.9126819126819127
random choice not knowing m success rate :  0.9433962264150944
mean HLG grouped :  260.99133333333333
mean LLG grouped :  9258.004


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)


In [ ]:
#obtain attack results for 1000 iterations with k=1000

hard_chosen=[]
secretary_chosen=[]
secretary_success=[]
fr=[]
shots=1000
iter=1000


for i in range(iter):
  generate_stream_data()
  onlineAttack()
  plot()



fr=np.asarray(fr, dtype=np.float32)
secretary_success=np.asarray(secretary_success, dtype=np.float32)

secretary_chosen=np.asarray(secretary_chosen, dtype=np.float32)
hard_chosen=np.asarray(hard_chosen, dtype=np.float32)

print("These are general results yielded for ",iter," number of random permutations")
print("average fool rate under my assumption : ",np.mean(fr))

print("average success for secretary chosen data : ",np.mean(secretary_success))
print("average number of data chosen by secretary : ",np.mean(secretary_chosen))
print("average number of hard chosen data : ",np.mean(hard_chosen))
print("average number of selected data : ",np.mean(hard_chosen+secretary_chosen))
print("random MLG success rate : ",randomSelectionInMLG())
print("random choice not knowing m success rate : ",randomSelectionNotKnowingM())
print("mean HLG grouped : ",np.mean(np.asarray(HLGRatio)))
print("mean LLG grouped : ",np.mean(np.asarray(LLGRatio)))


hard_chosen=[]
secretary_chosen=[]

secretary_success=[]
fr=[]







def plotShots():
  
  for i in indices:
    x= np.expand_dims(x_test[i], axis=0)
    yc=model(x)
    perturbation=fgsm(epsilon,i)
    print("perturbation shape ",perturbation.shape)
    print("x",x.shape)
    perturbation=np.expand_dims(perturbation,axis=(0,-1))
    xp=np.squeeze(perturbation)+np.squeeze(x)
    xp=np.expand_dims(xp,axis=(0,-1))
  
    yp=model(xp)

    if (np.argmax(yc)!=np.argmax(yp)) and (result[i] != -1):
      testImage(np.squeeze(x),np.squeeze(xp),np.argmax(yc),np.argmax(yp))

#plotShots()



These are general results yielded for  1000  number of random permutations
average fool rate under my assumption :  0.38954097
average success for secretary chosen data :  0.9892677
average number of data chosen by secretary :  130.109
average number of hard chosen data :  260.34
average number of selected data :  390.449
random MLG success rate :  0.9126819126819127
random choice not knowing m success rate :  0.9433962264150944
mean HLG grouped :  260.991
mean LLG grouped :  9258.0035
